<a href="https://colab.research.google.com/github/charlie9526/TaxiFare/blob/master/final_sumbission_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 . Mounting

In [33]:
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/Colab\ Notebooks/FAIR_TAXI

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/FAIR_TAXI


# 2 . Imports

In [0]:
import pandas as pd
from datetime import datetime
import random
from google.colab import files
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)              
import numpy as np                 
from scipy.stats import norm   
import warnings
warnings.filterwarnings('ignore')   
%matplotlib inline                  
from math import sin, cos, sqrt, atan2, radians
import math

from sklearn.cluster import KMeans

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score,fbeta_score,f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC,LinearSVC
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from xgboost import XGBClassifier
from sklearn.cluster import KMeans



# 3 . Read files

In [0]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# 4 . Functions

In [0]:

# set distance column
class set_distance(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
      return self

    def transform(self, X):
      X['distance'] = X.apply(lambda x:get_distance(x), axis=1)
      # print(X)
      # X = X.drop(labels=['pick_lat','pick_lon','drop_lat','drop_lon'],axis=1)
      return X
# lambda column get distance
def get_distance(x):
  R = 6373.0
  lat1 = radians(float(x['drop_lat']))
  lon1 = radians(float(x['drop_lon']))
  lat2 = radians(float(x['pick_lat']))
  lon2 = radians(float(x['pick_lon']))

  dlon = lon2 - lon1
  dlat = lat2 - lat1

  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  distance = round( R * c,2)
  return distance


def encoding_label(label):
  if(label=='correct'):
    return 1
  elif(label=='incorrect'):
    return 0
  else:
    print ("invalid label")

def get_hour_value(x,date_time_column_name):
  if x[date_time_column_name]==None:
    return None
  h =  datetime.strptime(x[date_time_column_name],"%m/%d/%Y %H:%M").hour
  return h

def get_minute_value(x,date_time_column_name):
  if x[date_time_column_name]==None:
    return None
  h =  datetime.strptime(x[date_time_column_name],"%m/%d/%Y %H:%M").minute
  return h


def get_fare_per_hr(x):
  if  x["duration"]>0:
    hours = x["duration"]
    val = float(x['fare'])/hours
    return val
  if x["duration"]==0:
    return 0
  return None


def get_addfare_per_hr(x):
  if  x["distance"]>0:
    val = float(x['additional_fare'])/x["distance"]
    return val
  if x["distance"]==0:
    return 0
  return None


def get_dis_per_fare(x):
  if x['fare']==0:
    return 0
  if x['fare']>0:
    return float(x['distance'])/x['fare']
  else:
    return 0


def get_waiting_rate(x):
  if math.isnan(x['meter_waiting']):
    return None
  if x['meter_waiting']==0:
    return 0
  if  x['meter_waiting_fare']>=0:
    return float(x['meter_waiting_fare'])/x['meter_waiting']
  else:
    return 0


def get_velocity(x):
  if math.isnan(x['duration']) or math.isnan(x['distance']) or math.isnan(x['meter_waiting']):
    return None
  if (x['duration']-x['meter_waiting'])==0:
    return 0
  return (float(x['distance']))/(x['duration']-x['meter_waiting'])


def get_mw_ratio(x):
  if (x['duration'])==0:
    return 0
  return float(x['meter_waiting'])/(x['duration'])


def get_mwfare_ratio(x):
  if (x['fare'])==0:
    return 0
  return float(x['meter_waiting_fare'])/(x['fare'])



def invget_fare_per_hr(x):
  if  x["fare"]>0:
    fare = x["duration"]
    val = float(x['fare'])/fare
    return val
  if x["fare"]==0:
    return 0
  return None


# 5 . PreProcess

### Some pre steps


In [0]:
# train_df = train_df.drop_duplicates()
# train_df = train_df.dropna()
################# ugly indexes
# bads = [203757140,213112837,193653017,209365576,190167541]
# train_df = train_df[~(train_df['tripid'].isin(bads))]

### Create feature_df and label

In [0]:
trip_id  = test_df['tripid']

features_df = train_df.loc[:, train_df.columns != 'label']
label_df = train_df['label'].apply(encoding_label).values

features_df = features_df.loc[:, features_df.columns != 'tripid']
test_df = test_df.loc[:, test_df.columns != 'tripid']

### New columns

In [0]:
features_df['pickup_hour'] = features_df.apply(lambda x : get_hour_value(x,'pickup_time'),axis=1)
test_df['pickup_hour'] = test_df.apply(lambda x : get_hour_value(x,'pickup_time'),axis=1)

features_df['pickup_minute'] = features_df.apply(lambda x : get_minute_value(x,'pickup_time'),axis=1)
test_df['pickup_minute'] = test_df.apply(lambda x : get_minute_value(x,'pickup_time'),axis=1)

features_df['drop_hour'] = features_df.apply(lambda x : get_hour_value(x,'drop_time'),axis=1)
test_df['drop_hour'] = test_df.apply(lambda x : get_hour_value(x,'drop_time'),axis=1)

features_df['drop_minute'] = features_df.apply(lambda x : get_minute_value(x,'drop_time'),axis=1)
test_df['drop_minute'] = test_df.apply(lambda x : get_minute_value(x,'drop_time'),axis=1)

features_df['distance'] = features_df.apply(lambda x:get_distance(x), axis=1)
test_df['distance'] = test_df.apply(lambda x:get_distance(x), axis=1)

features_df['fare_per_time'] = features_df['fare']/(features_df['duration'])
test_df['fare_per_time'] = test_df['fare']/(test_df['duration'])

features_df['run_time_per_fare'] = (features_df['duration']-features_df['meter_waiting'])/(features_df['fare'])
test_df['run_time_per_fare'] = (test_df['duration']-test_df['meter_waiting'])/(test_df['fare'])

features_df['run_time_per_run_fare'] = (features_df['duration']-features_df['meter_waiting'])/(features_df['fare']-features_df['meter_waiting_fare'])
test_df['run_time_per_run_fare'] = (test_df['duration']-test_df['meter_waiting'])/(test_df['fare']-test_df['meter_waiting_fare'])

features_df['add_fare_per_time'] = (features_df['additional_fare'])/(features_df['duration'])
test_df['add_fare_per_time'] = (test_df['additional_fare'])/(test_df['duration'])

features_df['distance_per_run_time'] = (features_df['duration'])/(features_df['distance'])
test_df['distance_per_run_time'] = (test_df['duration'])/(test_df['distance'])

features_df['waiting_ratio'] = (features_df['duration'])/(features_df['meter_waiting'])
test_df['waiting_ratio'] = (test_df['duration'])/(test_df['meter_waiting'])

features_df['fare_ratio'] = (features_df['additional_fare']+features_df['meter_waiting_fare'])/(features_df['fare'])
test_df['fare_ratio'] = (test_df['additional_fare']+test_df['meter_waiting_fare'])/(test_df['fare'])

features_df['realtime_ratio'] = (features_df['duration']-features_df['meter_waiting'])/(features_df['duration'])
test_df['realtime_ratio'] = (test_df['duration']-test_df['meter_waiting'])/(test_df['duration'])

features_df['fare_per_waiting'] = (features_df['meter_waiting_fare'])/(features_df['meter_waiting'])
test_df['fare_per_waiting'] = (test_df['meter_waiting_fare'])/(test_df['meter_waiting'])





### Drop Columns

In [0]:
feature_list = [
                'pickup_hour',                                
                'pickup_minute',
                'distance',
                'fare_per_time',
                'run_time_per_fare',
                'run_time_per_run_fare',
                'add_fare_per_time',
                'distance_per_run_time',
                'waiting_ratio',
                'fare_ratio',
                'fare_per_waiting',
                'additional_fare',
                'duration',
                'meter_waiting',
                'meter_waiting_fare',
                'fare'
                ]

features_df = features_df[feature_list]
test_df = test_df[feature_list]


# 7 . Cross validation and result

In [41]:
# for x in np.arange(0.05, 1, 0.01):
for x in range(0,1,1):
  xgb = XGBClassifier(
      learning_rate=0.19,
  )

  kfold = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
  cv_results = cross_val_score(xgb, features_df, label_df, cv=kfold, scoring='f1')
  # print(cv_results)
  print('%s: %f (%f) for rs = %f ' % ("F1", cv_results.mean()*100, cv_results.std(),x))
  cv_results = cross_val_score(xgb, features_df, label_df, cv=kfold, scoring='precision')
  # print(cv_results)
  print('%s: %f (%f) for rs = %f ' % ("precision", cv_results.mean()*100, cv_results.std(),x))
  cv_results = cross_val_score(xgb, features_df, label_df, cv=kfold, scoring='recall')
  # print(cv_results)
  print('%s: %f (%f) for rs = %f ' % ("recall", cv_results.mean()*100, cv_results.std(),x))
  cv_results = cross_val_score(xgb, features_df, label_df, cv=kfold, scoring='accuracy')
  # print(cv_results)
  print('%s: %f (%f) for rs = %f ' % ("accuracy", cv_results.mean()*100, cv_results.std(),x))

  print("\n")

F1: 97.652284 (0.003237) for rs = 0.000000 
precision: 96.427330 (0.004071) for rs = 0.000000 
recall: 98.909353 (0.003113) for rs = 0.000000 
accuracy: 95.709199 (0.005944) for rs = 0.000000 




In [0]:
# F1: 97.619646 (0.003619) for rs = 0.000000 
# precision: 96.418998 (0.004029) for rs = 0.000000 
# recall: 98.851296 (0.004098) for rs = 0.000000 
# accuracy: 95.651002 (0.006617) for rs = 0.000000 



# 8 . Train model

In [43]:
xgb.fit(features_df,label_df)
pd.DataFrame({'feature':feature_list,'importance':xgb.feature_importances_}).sort_values(by=['importance'])

,feature,importance
1,pickup_minute,0.008481
0,pickup_hour,0.011381
6,add_fare_per_time,0.017678
9,fare_ratio,0.018753
14,meter_waiting_fare,0.019034
8,waiting_ratio,0.019708
2,distance,0.025548
13,meter_waiting,0.026909
10,fare_per_waiting,0.031991
12,duration,0.035664


# 9 . Predict and save result

In [0]:
predicted = xgb.predict(test_df)

data_dict = { 'tripid':trip_id, 'prediction':predicted}
pd_test_result = pd.DataFrame(data=data_dict,columns=['tripid','prediction'])
pd_test_result = pd_test_result.set_index('tripid') 
pd_test_result.to_csv("final_sumbission_1_.csv", index=True)

In [45]:
pd_test_result['prediction'].value_counts()

1    8156
0     420
Name: prediction, dtype: int64